In [13]:
import random
from collections import Counter

import numpy as np
import pandas as pd
from PIL import Image

#### Конвертируем картинку сначала в чб с градацией серого. Потом только в черное и белое.
Вариант использовать bw = input_image.convert('1') мне не очень нравится, поэтому оставил так.

In [14]:
def black_and_white(input_image: Image, divider=100):
    width, height = input_image.size
    bw = input_image.convert('L')
    pixels = bw.load()
    for x in range(width):
        for y in range(height):
            if pixels[x, y] > divider:
                pixels[x, y] = 255
            else:
                pixels[x, y] = 0
    return bw

Скейл картинки в 2 раза

In [15]:
def upscale(input_image: Image):
    width, height = input_image.size
    s_width = input_image.size[0] * 2 - 1
    s_height = input_image.size[1] * 2 - 1

    scale_img = Image.new("RGB", (s_width, s_height), 'white')
    img_pixels = input_image.load()
    scale_img_pixels = scale_img.load()

    for x in range(width):
        for y in range(height):
            scale_img_pixels[x * 2, y * 2] = img_pixels[x, y]

    for x in range(1, s_width, 2):
        for y in range(1, s_height, 2):
            p1 = scale_img_pixels[x - 1, y - 1]
            p2 = scale_img_pixels[x - 1, y + 1]
            p3 = scale_img_pixels[x + 1, y - 1]
            p4 = scale_img_pixels[x + 1, y + 1]
            p = []

            for i in range(3):
                p.append((p1[i] + p2[i] + p3[i] + p4[i]) // 4)

            scale_img_pixels[x, y] = tuple(p)

    for x in range(0, s_width, 1):
        for y in range(0, s_height, 1):
            if (x + y) % 2 != 0:
                p_near = []
                pix_r = 0
                pix_g = 0
                pix_b = 0

                for x_i in range(x - 1, x + 2, 1):
                    for y_i in range(y - 1, y + 2, 1):
                        if 0 <= x_i < s_width and 0 <= y_i < s_height:
                            if abs(x_i + y_i - x - y) == 1:
                                p_near.append(scale_img_pixels[x_i, y_i])

                for p in p_near:
                    pix_r += p[0]
                    pix_g += p[1]
                    pix_b += p[2]

                if len(p_near) > 0:
                    scale_img_pixels[x, y] = (pix_r // len(p_near), pix_g // len(p_near), pix_b // len(p_near))
                else:
                    scale_img_pixels[x, y] = (255, 255, 255)

    return scale_img

Просто удобно вызывать одну функцию. Данные: норм и косячные

In [16]:
def draw_assumption(data):
    df = pd.DataFrame(data[0])
    df.columns = indexes
    print(df)

    if data[1]:
        print('-' * 100)
        ejection = pd.DataFrame(data[1])
        ejection.columns = indexes
        print(ejection)
        print("=" * 100)


def check_results(*data_list):
    equals = True
    if len(data_list) < 2:
        return
    for i in range(1, len(data_list)):
        if data_list[0] != data_list[i]:
            equals = False
    print("Equals =", equals)
    draw_assumption(data_list[0])
    if not equals:
        for i in range(1, len(data_list)):
            draw_assumption(data_list[i])

In [17]:
def k_means(noised_image: Image):
    noised_image = noised_image.convert('RGB')
    width, height = noised_image.size
    min_dist = 255 * 255 * 255 * width * height
    noised_image_pixels = noised_image.load()
    nearest_number = 0

    for n in range(10):
        number = Image.open(f'../resources/numbers/{width}x{height}/{n}.png')
        width, height = number.size
        number_pixels = number.load()
        tmp_dist = 0

        for x in range(width):
            for y in range(height):
                if number_pixels[x, y] != noised_image_pixels[x, y]:
                    rgb = 0
                    for i in range(3):
                        rgb += abs(noised_image_pixels[x, y][i] - number_pixels[x, y][i])
                    tmp_dist += (rgb / 3)
        if min_dist > tmp_dist:
            nearest_number = n
            min_dist = tmp_dist

    return nearest_number


def k_means_ejections(noised_image: Image, max_dist = 4000):
    noised_image = noised_image.convert('RGB')
    width, height = noised_image.size
    min_dist = 255 * 255 * 255 * width * height
    noised_image_pixels = noised_image.load()
    nearest_number = 0

    for n in range(10):
        number = Image.open(f'../resources/numbers/{width}x{height}/{n}.png')
        width, height = number.size
        number_pixels = number.load()
        tmp_dist = 0

        for x in range(width):
            for y in range(height):
                if number_pixels[x, y] != noised_image_pixels[x, y]:
                    rgb = 0
                    for i in range(3):
                        rgb += abs(noised_image_pixels[x, y][i] - number_pixels[x, y][i])
                    tmp_dist += (rgb / 3)
        if min_dist > tmp_dist:
            nearest_number = n
            min_dist = tmp_dist

    if min_dist > max_dist:
        return {"distance": min_dist, "nearest_number": nearest_number}


def k_means_scaled(noised_image: Image):
    noised_image = noised_image.convert('RGB')
    width, height = noised_image.size
    min_dist = 255 * 255 * 255 * width * height
    noised_image_pixels = noised_image.load()
    nearest_number = 0

    for n in range(10):
        input_number = Image.open(f'../resources/numbers/{7}x{8}/{n}.png')
        number = input_number.copy()
        while width > number.width and height > number.height:
            number = upscale(number)
        width, height = number.size
        number_pixels = number.load()
        tmp_dist = 0

        for x in range(width):
            for y in range(height):
                if number_pixels[x, y] != noised_image_pixels[x, y]:
                    rgb = 0
                    for i in range(3):
                        rgb += abs(noised_image_pixels[x, y][i] - number_pixels[x, y][i])
                    tmp_dist += (rgb / 3)
        if min_dist > tmp_dist:
            nearest_number = n
            min_dist = tmp_dist

    return nearest_number


def k_means_for_bw(noised_image: Image):
    noised_image = black_and_white(noised_image)
    width, height = noised_image.size
    min_dist = width * height
    noised_image_pixels = noised_image.load()
    nearest_number = 0

    for n in range(10):
        number = black_and_white(Image.open(f'../resources/numbers/{width}x{height}/{n}.png'))
        width, height = number.size
        number_pixels = number.load()
        tmp_dist = 0

        for x in range(width):
            for y in range(height):
                if number_pixels[x, y] != noised_image_pixels[x, y]:
                    tmp_dist += 1

        if min_dist > tmp_dist:
            nearest_number = n
            min_dist = tmp_dist

    return nearest_number


def k_means_for_grey(noised_image: Image):
    noised_image = noised_image.convert('L')
    width, height = noised_image.size
    min_dist = width * height * 255
    noised_image_pixels = noised_image.load()
    nearest_number = 0

    for n in range(10):
        number = black_and_white(Image.open(f'../resources/numbers/{width}x{height}/{n}.png'))
        width, height = number.size
        number_pixels = number.load()
        tmp_dist = 0

        for x in range(width):
            for y in range(height):
                if number_pixels[x, y] != noised_image_pixels[x, y]:
                    tmp_dist += abs(noised_image_pixels[x, y] - number_pixels[x, y])

        if min_dist > tmp_dist:
            nearest_number = n
            min_dist = tmp_dist

    return nearest_number

#### Шумы для цветной картинки

In [18]:
def noise_image(input_image: Image, probability: float):
    if probability > 1 or probability < 0:
        probability = random.random()

    input_image = input_image.copy()
    width, height = input_image.size
    pixels = input_image.load()
    p = random.uniform(0, probability)
    for x in range(width):
        for y in range(height):
            if random.random() < p:
                pixels[x, y] = tuple(np.random.choice(range(256), size=3))

    return input_image


def noise_area_image(input_image: Image, center: (int, int), radius: int, probability: float):
    input_image = input_image.copy()
    width, height = input_image.size
    pixels = input_image.load()
    p = random.uniform(0, probability)

    x_start = 0
    y_start = 0
    x_finish = width
    y_finish = height
    if center[0] - radius > 0:
        x_start = center[0] - radius
    if center[1] - radius > 0:
        y_start = center[1] - radius
    if center[0] + radius < width:
        x_finish = center[0] + radius
    if center[1] + radius < height:
        y_finish = center[1] + radius

    for x in range(x_start, x_finish):
        for y in range(y_start, y_finish):
            if random.random() < p:
                pixels[x, y] = tuple(np.random.choice(range(256), size=3))
    return input_image


#### Шумы для чб картинки.

In [19]:
def noise_bw_image(input_image: Image, probability: float):
    if probability > 1 or probability < 0:
        probability = random.random()

    input_image = input_image.copy()
    input_image = black_and_white(input_image)
    width, height = input_image.size
    pixels = input_image.load()
    p = random.uniform(0, probability)

    for x in range(width):
        for y in range(height):
            if random.random() < p:
                if pixels[x, y] == 0:
                    pixels[x, y] = 255
                else:
                    pixels[x, y] = 0
    return input_image


def noise_bw_area_image(input_image: Image, center: (int, int), radius: int, probability: float):
    input_image = input_image.copy()
    width, height = input_image.size
    pixels = input_image.load()
    p = random.uniform(0, probability)

    x_start = 0
    y_start = 0
    x_finish = width
    y_finish = height
    if center[0] - radius > 0:
        x_start = center[0] - radius
    if center[1] - radius > 0:
        y_start = center[1] - radius
    if center[0] + radius < width:
        x_finish = center[0] + radius
    if center[1] + radius < height:
        y_finish = center[1] + radius

    for x in range(x_start, x_finish):
        for y in range(y_start, y_finish):
            if random.random() < p:
                if pixels[x, y] == 0:
                    pixels[x, y] = 255
                else:
                    pixels[x, y] = 0
    return input_image


#### Шумы для картинки с градацией серого

In [20]:
def noise_grey_image(input_image: Image, probability: float):
    if probability > 1 or probability < 0:
        probability = random.random()

    input_image = input_image.copy()
    input_image = input_image.convert("L")
    width, height = input_image.size
    pixels = input_image.load()
    p = random.uniform(0, probability)

    for x in range(width):
        for y in range(height):
            if random.random() < p:
                pixels[x, y] = random.randint(0, 255)
    return input_image


def noise_grey_area_image(input_image: Image, center: (int, int), radius: int, probability: float):
    input_image = input_image.copy()
    width, height = input_image.size
    pixels = input_image.load()
    p = random.uniform(0, probability)

    x_start = 0
    y_start = 0
    x_finish = width
    y_finish = height
    if center[0] - radius > 0:
        x_start = center[0] - radius
    if center[1] - radius > 0:
        y_start = center[1] - radius
    if center[0] + radius < width:
        x_finish = center[0] + radius
    if center[1] + radius < height:
        y_finish = center[1] + radius

    for x in range(x_start, x_finish):
        for y in range(y_start, y_finish):
            if random.random() < p:
                pixels[x, y] = random.randint(0, 255)
    return input_image



In [21]:
indexes = list(np.arange(0.3, 0.75, 0.05))


def generate_noise_numbers(width, height, noise_func, tries_per_noise):
    res = []
    for n in range(10):
        number = Image.open(f'../resources/numbers/{width}x{height}/{n}.png')
        tmp = []
        for noise in indexes:
            noise_list = []
            for i in range(tries_per_noise):
                noised_number = noise_func(number, noise)
                noise_list.append(noised_number)
            tmp.append(noise_list)
        res.append(tmp)
    return res


def predict(numbers, assumption_func):
    data = []
    wrong_data = []
    success_predicted = 0
    correct_number = 0
    is_empty = True

    for n in numbers:
        wrong_predicted = []
        tmp = []
        for noise in n:
            wrong_assumption = []
            for number in noise:
                assumption = assumption_func(number)
                if correct_number == assumption:
                    success_predicted += 1
                else:
                    is_empty = False
                    wrong_assumption.append(assumption)
            wrong_predicted.append(Counter(wrong_assumption))
            tmp.append(success_predicted)
            success_predicted = 0
        data.append(tmp)
        wrong_data.append(wrong_predicted)
        correct_number += 1
    if is_empty:
        wrong_data = []
    return [data, wrong_data]

### Предсказываем для чисел размером 7 на 8

In [22]:
rgb_numbers = generate_noise_numbers(7, 8, lambda image, noise: noise_image(image, noise), 10)
bw_numbers = generate_noise_numbers(7, 8, lambda image, noise: noise_bw_image(image, noise), 10)
grey_numbers = generate_noise_numbers(7, 8, lambda image, noise: noise_grey_image(image, noise), 10)

In [23]:
res = []
for i in rgb_numbers:
    for j in i:
        for k in j:
            # print(k)
            r = k_means_ejections(k, 4500)
            if r:
                res.append(r)
print(f'{len(res)}/{len(indexes)*100}')
print(res)

4/900
[{'distance': 4538.000000000001, 'nearest_number': 1}, {'distance': 4716.999999999999, 'nearest_number': 6}, {'distance': 4782.333333333333, 'nearest_number': 6}, {'distance': 5330.666666666665, 'nearest_number': 7}]


In [24]:
check_results(predict(rgb_numbers, lambda noised_number: k_means(noised_number)),
              predict(rgb_numbers, lambda noised_number: k_means_for_grey(noised_number)),
              predict(rgb_numbers, lambda noised_number: k_means_for_bw(noised_number)))

Equals = False
   0.30  0.35  0.40  0.45  0.50  0.55  0.60  0.65  0.70
0    10    10    10    10    10    10    10    10    10
1    10    10    10    10    10    10    10    10    10
2    10    10    10    10    10    10    10    10    10
3    10    10    10    10    10    10    10    10    10
4    10    10    10    10    10    10    10    10    10
5    10    10    10    10    10    10    10    10    10
6    10    10    10    10    10    10    10    10    10
7    10    10    10    10    10    10    10    10    10
8    10    10    10    10    10    10    10    10    10
9    10    10    10    10    10    10    10    10    10
   0.30  0.35  0.40  0.45  0.50  0.55  0.60  0.65  0.70
0    10    10    10     9    10    10     9     9     9
1    10    10    10    10    10    10    10    10    10
2    10    10    10    10    10    10    10    10    10
3    10    10    10    10     9     9    10     9     9
4    10    10    10    10    10    10    10    10    10
5    10    10    10    10    10  

In [25]:
check_results(predict(grey_numbers, lambda noised_number: k_means_for_grey(noised_number)),
              predict(grey_numbers, lambda noised_number: k_means_for_bw(noised_number)))

Equals = False
   0.30  0.35  0.40  0.45  0.50  0.55  0.60  0.65  0.70
0    10    10    10    10    10    10     9     7    10
1    10    10    10    10    10    10    10    10    10
2    10    10    10    10    10    10    10     9    10
3    10    10    10    10    10     9    10     9     8
4    10    10    10    10    10    10    10    10    10
5    10    10    10    10    10    10    10    10     9
6    10    10    10    10    10    10    10    10     9
7    10    10    10    10    10    10    10    10    10
8    10    10     9    10    10     9    10    10    10
9    10    10    10     9    10    10    10     8     9
----------------------------------------------------------------------------------------------------
  0.30 0.35    0.40    0.45 0.50    0.55    0.60          0.65    0.70
0   {}   {}      {}      {}   {}      {}  {6: 1}  {9: 2, 6: 1}      {}
1   {}   {}      {}      {}   {}      {}      {}            {}      {}
2   {}   {}      {}      {}   {}      {}      {}       

In [26]:
draw_assumption(predict(bw_numbers, lambda noised_number: k_means_for_bw(noised_number)))

   0.30  0.35  0.40  0.45  0.50  0.55  0.60  0.65  0.70
0     9     9     8     9     9     5     5     5     3
1    10    10     9     8     9     9     7     7     7
2    10    10     7     7     8     5     7     5     4
3    10     9     9     8     6     6     7     5     6
4    10    10    10     9     7    10     6     6     5
5    10    10     9     7     6     3     6     7     5
6     8    10     7    10     9     5     7     4     5
7    10    10     8     7     9     4     6     5     6
8    10     9     7     8     4     5     4     9     6
9     9     8     7     8    10     8     3     5     6
----------------------------------------------------------------------------------------------------
     0.30          0.35                0.40                0.45  \
0  {5: 1}        {8: 1}        {9: 1, 8: 1}              {9: 1}   
1      {}            {}              {2: 1}        {3: 1, 7: 1}   
2      {}            {}        {3: 2, 0: 1}        {1: 2, 5: 1}   
3      {}      

В целом логично все, вроде. У нас в обычных случаях "точечные" ошибки.
Когда же мы скейлим, то они вносят свой "плохой" импакт для нормальных пикселей. Поэтому в итоге этот варик - шляпа.

In [27]:
rgb_numbers = generate_noise_numbers(7, 8, lambda image, noise: noise_image(image, noise), 10)
rgb_numbers_scaled = []
for n in rgb_numbers:
    tmp2 = []
    for number in n:
        tmp1 = []
        for image in number:
            tmp1.append(upscale(image))
        tmp2.append(tmp1)
    rgb_numbers_scaled.append(tmp2)

check_results(predict(rgb_numbers_scaled, lambda noised_number: k_means_scaled(noised_number)),
              predict(rgb_numbers, lambda noised_number: k_means(noised_number)))

Equals = False
   0.30  0.35  0.40  0.45  0.50  0.55  0.60  0.65  0.70
0    10    10    10    10    10    10    10    10    10
1    10    10    10    10    10    10    10    10    10
2    10    10    10    10    10    10    10    10    10
3    10    10    10    10    10    10     9    10     9
4    10    10    10    10    10    10    10    10    10
5    10    10    10    10    10    10    10    10    10
6    10    10    10    10    10    10    10    10     9
7    10    10    10    10    10     9     9    10    10
8    10    10    10    10    10    10    10    10    10
9    10    10    10    10    10    10    10    10     9
----------------------------------------------------------------------------------------------------
  0.30 0.35 0.40 0.45 0.50    0.55    0.60 0.65    0.70
0   {}   {}   {}   {}   {}      {}      {}   {}      {}
1   {}   {}   {}   {}   {}      {}      {}   {}      {}
2   {}   {}   {}   {}   {}      {}      {}   {}      {}
3   {}   {}   {}   {}   {}      {}  {9: 1}  

#### Смотрим как схавает шум по области

In [28]:
x_rand = random.randint(0, 7)
y_rand = random.randint(0, 8)
r_list = list(np.arange(3, 6, 1))
print(x_rand, y_rand, r_list)

rgb_numbers = []
bw_numbers = []
grey_numbers = []
for r in r_list:
    rgb_numbers.append(generate_noise_numbers(7, 8,
                                              lambda image, noise: noise_area_image(image, (x_rand, y_rand), r, noise),
                                              10))
    bw_numbers.append(generate_noise_numbers(7, 8,
                                             lambda image, noise: noise_bw_area_image(image, (x_rand, y_rand), r,
                                                                                      noise), 10))
    grey_numbers.append(generate_noise_numbers(7, 8,
                                               lambda image, noise: noise_grey_area_image(image, (x_rand, y_rand), r,
                                                                                          noise), 10))

1 4 [3, 4, 5]


In [29]:
print(r_list)
for number in rgb_numbers:
    check_results(predict(number, lambda noised_number: k_means(noised_number)),
                  predict(number, lambda noised_number: k_means_for_grey(noised_number)),
                  predict(number, lambda noised_number: k_means_for_bw(noised_number)))

[3, 4, 5]
Equals = True
   0.30  0.35  0.40  0.45  0.50  0.55  0.60  0.65  0.70
0    10    10    10    10    10    10    10    10    10
1    10    10    10    10    10    10    10    10    10
2    10    10    10    10    10    10    10    10    10
3    10    10    10    10    10    10    10    10    10
4    10    10    10    10    10    10    10    10    10
5    10    10    10    10    10    10    10    10    10
6    10    10    10    10    10    10    10    10    10
7    10    10    10    10    10    10    10    10    10
8    10    10    10    10    10    10    10    10    10
9    10    10    10    10    10    10    10    10    10
Equals = False
   0.30  0.35  0.40  0.45  0.50  0.55  0.60  0.65  0.70
0    10    10    10    10    10    10    10    10    10
1    10    10    10    10    10    10    10    10    10
2    10    10    10    10    10    10    10    10    10
3    10    10    10    10    10    10    10    10    10
4    10    10    10    10    10    10    10    10    10
5    10  

In [30]:
print(r_list)
for number in grey_numbers:
    check_results(predict(number, lambda noised_number: k_means_for_grey(noised_number)),
                  predict(number, lambda noised_number: k_means_for_bw(noised_number)))

[3, 4, 5]
Equals = False
   0.30  0.35  0.40  0.45  0.50  0.55  0.60  0.65  0.70
0    10    10    10    10    10    10    10    10    10
1    10    10    10    10    10    10    10    10    10
2    10    10    10    10    10    10    10    10    10
3     9    10    10    10    10    10     9    10     8
4    10    10    10    10    10    10    10    10    10
5    10    10    10    10    10    10    10    10    10
6    10    10    10    10    10    10    10    10    10
7    10    10    10    10    10    10    10    10    10
8    10    10    10    10    10    10    10    10    10
9    10    10    10    10    10    10    10    10    10
----------------------------------------------------------------------------------------------------
     0.30 0.35 0.40 0.45 0.50 0.55    0.60 0.65    0.70
0      {}   {}   {}   {}   {}   {}      {}   {}      {}
1      {}   {}   {}   {}   {}   {}      {}   {}      {}
2      {}   {}   {}   {}   {}   {}      {}   {}      {}
3  {8: 1}   {}   {}   {}   {}   {}

In [31]:
print(r_list)
for number in bw_numbers:
    draw_assumption(predict(number, lambda noised_number: k_means_for_bw(noised_number)))

[3, 4, 5]
   0.30  0.35  0.40  0.45  0.50  0.55  0.60  0.65  0.70
0    10    10    10    10    10    10    10    10    10
1    10    10    10    10    10    10    10    10    10
2    10    10    10    10    10    10    10    10    10
3    10    10    10    10    10    10    10    10    10
4    10    10    10    10    10    10    10    10    10
5    10    10    10    10    10    10    10    10    10
6    10    10    10    10    10    10    10    10    10
7    10    10    10    10    10    10    10    10    10
8    10    10    10    10    10    10    10    10    10
9    10    10    10    10    10    10    10    10    10
   0.30  0.35  0.40  0.45  0.50  0.55  0.60  0.65  0.70
0    10    10     9    10    10    10     9    10    10
1    10    10    10    10    10    10    10    10    10
2    10    10    10    10    10    10    10    10    10
3    10    10    10    10    10    10    10     8     7
4    10    10    10    10    10    10    10    10    10
5    10    10    10    10    10    10 

### Предсказываем для чисел размером 55 на 78

In [32]:
rgb_numbers = generate_noise_numbers(55, 78, lambda image, noise: noise_image(image, noise), 1)
bw_numbers = generate_noise_numbers(55, 78, lambda image, noise: noise_bw_image(image, noise), 1)
grey_numbers = generate_noise_numbers(55, 78, lambda image, noise: noise_grey_image(image, noise), 1)

In [33]:
check_results(predict(rgb_numbers, lambda noised_number: k_means(noised_number)),
              predict(rgb_numbers, lambda noised_number: k_means_for_grey(noised_number)),
              predict(rgb_numbers, lambda noised_number: k_means_for_bw(noised_number)))

Equals = True
   0.30  0.35  0.40  0.45  0.50  0.55  0.60  0.65  0.70
0     1     1     1     1     1     1     1     1     1
1     1     1     1     1     1     1     1     1     1
2     1     1     1     1     1     1     1     1     1
3     1     1     1     1     1     1     1     1     1
4     1     1     1     1     1     1     1     1     1
5     1     1     1     1     1     1     1     1     1
6     1     1     1     1     1     1     1     1     1
7     1     1     1     1     1     1     1     1     1
8     1     1     1     1     1     1     1     1     1
9     1     1     1     1     1     1     1     1     1


In [34]:
check_results(predict(grey_numbers, lambda noised_number: k_means_for_grey(noised_number)),
              predict(grey_numbers, lambda noised_number: k_means_for_bw(noised_number)))

Equals = True
   0.30  0.35  0.40  0.45  0.50  0.55  0.60  0.65  0.70
0     1     1     1     1     1     1     1     1     1
1     1     1     1     1     1     1     1     1     1
2     1     1     1     1     1     1     1     1     1
3     1     1     1     1     1     1     1     1     1
4     1     1     1     1     1     1     1     1     1
5     1     1     1     1     1     1     1     1     1
6     1     1     1     1     1     1     1     1     1
7     1     1     1     1     1     1     1     1     1
8     1     1     1     1     1     1     1     1     1
9     1     1     1     1     1     1     1     1     1


In [35]:
draw_assumption(predict(bw_numbers, lambda noised_number: k_means_for_bw(noised_number)))

   0.30  0.35  0.40  0.45  0.50  0.55  0.60  0.65  0.70
0     1     1     1     1     1     1     1     1     1
1     1     1     1     1     1     1     0     1     1
2     1     1     1     1     1     1     1     1     1
3     1     1     1     1     1     1     1     1     1
4     1     1     1     1     1     1     1     1     1
5     1     1     1     1     1     1     1     1     0
6     1     1     1     1     1     1     1     1     0
7     1     1     1     1     1     1     1     1     0
8     1     1     1     1     1     1     0     1     1
9     1     1     1     1     0     0     1     1     0
----------------------------------------------------------------------------------------------------
  0.30 0.35 0.40 0.45    0.50    0.55    0.60 0.65    0.70
0   {}   {}   {}   {}      {}      {}      {}   {}      {}
1   {}   {}   {}   {}      {}      {}  {2: 1}   {}      {}
2   {}   {}   {}   {}      {}      {}      {}   {}      {}
3   {}   {}   {}   {}      {}      {}      {}  

In [36]:
x_rand = random.randint(0, 55)
y_rand = random.randint(0, 78)
r_rand = 10
print(x_rand, y_rand)

rgb_numbers = generate_noise_numbers(7, 8,
                                     lambda image, noise: noise_area_image(image, (x_rand, y_rand), r_rand, noise), 1)
bw_numbers = generate_noise_numbers(7, 8,
                                    lambda image, noise: noise_bw_area_image(image, (x_rand, y_rand), r_rand, noise),
                                    1)
grey_numbers = generate_noise_numbers(7, 8, lambda image, noise: noise_grey_area_image(image, (x_rand, y_rand), r_rand,
                                                                                       noise), 1)

53 18


In [37]:
check_results(predict(rgb_numbers, lambda noised_number: k_means(noised_number)),
              predict(rgb_numbers, lambda noised_number: k_means_for_grey(noised_number)),
              predict(rgb_numbers, lambda noised_number: k_means_for_bw(noised_number)))

Equals = True
   0.30  0.35  0.40  0.45  0.50  0.55  0.60  0.65  0.70
0     1     1     1     1     1     1     1     1     1
1     1     1     1     1     1     1     1     1     1
2     1     1     1     1     1     1     1     1     1
3     1     1     1     1     1     1     1     1     1
4     1     1     1     1     1     1     1     1     1
5     1     1     1     1     1     1     1     1     1
6     1     1     1     1     1     1     1     1     1
7     1     1     1     1     1     1     1     1     1
8     1     1     1     1     1     1     1     1     1
9     1     1     1     1     1     1     1     1     1


In [38]:
check_results(predict(grey_numbers, lambda noised_number: k_means_for_grey(noised_number)),
              predict(grey_numbers, lambda noised_number: k_means_for_bw(noised_number)))

Equals = True
   0.30  0.35  0.40  0.45  0.50  0.55  0.60  0.65  0.70
0     1     1     1     1     1     1     1     1     1
1     1     1     1     1     1     1     1     1     1
2     1     1     1     1     1     1     1     1     1
3     1     1     1     1     1     1     1     1     1
4     1     1     1     1     1     1     1     1     1
5     1     1     1     1     1     1     1     1     1
6     1     1     1     1     1     1     1     1     1
7     1     1     1     1     1     1     1     1     1
8     1     1     1     1     1     1     1     1     1
9     1     1     1     1     1     1     1     1     1


In [39]:
draw_assumption(predict(bw_numbers, lambda noised_number: k_means_for_bw(noised_number)))

   0.30  0.35  0.40  0.45  0.50  0.55  0.60  0.65  0.70
0     1     1     1     1     1     1     1     1     1
1     1     1     1     1     1     1     1     1     1
2     1     1     1     1     1     1     1     1     1
3     1     1     1     1     1     1     1     1     1
4     1     1     1     1     1     1     1     1     1
5     1     1     1     1     1     1     1     1     1
6     1     1     1     1     1     1     1     1     1
7     1     1     1     1     1     1     1     1     1
8     1     1     1     1     1     1     1     1     1
9     1     1     1     1     1     1     1     1     1


## Пошумим!!!

In [40]:
def generate_noise(mode, size, color, probability, noise_func):
    return noise_func(Image.new(mode, size, color), probability)

In [41]:
img_list = []
for l in range(10):
    for i in indexes:
        img_list.append(generate_noise("RGB", (7, 8), 'white', i,
                             lambda noised_image, probability: noise_image(noised_image, probability)))


def distance(img1: Image, img2: Image):
    if img2.size != img1.size:
        return
    width, height = img2.size
    img1_pix = img1.load()
    img2_pix = img2.load()
    dist = 0

    for x in range(width):
            for y in range(height):
                if img1_pix[x, y] != img2_pix[x, y]:
                    rgb = 0
                    for i in range(3):
                        rgb += abs(img2_pix[x, y][i] - img1_pix[x, y][i])
                    dist += (rgb / 3)
    return dist

def odin(images_list: list, bound = 4, number_of_occurrences = 1):
    res = []
    for number in images_list:
        tmp_list = images_list.copy()
        tmp_list.remove(number)
        tmp_list.sort(key=lambda img: distance(number, img))
        res.append(tmp_list[:bound])

    t = []
    for n in res:
        for number in n:
            t.append(number)


    out = 0
    for number in images_list:
        # print(t.count(number))
        if t.count(number) <= number_of_occurrences:
            out +=1

    return out



print(f'out: {odin(img_list, 10, 2)}/{len(img_list)}')

out: 55/90


In [42]:
for i in indexes:
    img = generate_noise("1", (7, 8), 'white', i,
                         lambda noised_image, probability: noise_bw_image(noised_image, probability))
    # img.show()
    print(k_means_for_bw(img), k_means_for_grey(img), k_means(img))

7 7 1
4 4 7
6 6 6
4 4 7
4 4 7
0 0 7
1 1 1
4 4 4
4 4 1


In [43]:
for i in indexes:
    img = generate_noise("L", (7, 8), 'white', i,
                         lambda noised_image, probability: noise_bw_image(noised_image, probability))
    # img.show()
    print(k_means_for_bw(img), k_means_for_grey(img), k_means(img))

2 2 2
7 7 7
7 7 1
0 0 7
4 4 4
7 7 7
7 7 7
3 3 1
7 7 7


In [44]:
# img = Image.open('image0_1.jpg')
# upscale(img).save('image0_2.jpg')